In [321]:
import phonenumbers
import pycountry
from phonenumbers.phonenumberutil import (
    region_code_for_country_code,
    region_code_for_number,
)
import pandas as pd

# Read in all country code/phone number data
-- source: https://en.wikipedia.org/wiki/List_of_mobile_telephone_prefixes_by_country

In [322]:
# # filter for wh and only countries of interest
# data = pd.read_csv('table-1.csv')
# wh_data = data[data.Americas == True]

In [323]:
wh_phone_dict = pd.read_csv('wh_phone_dict.csv')
wh_phone_dict

,Country,Americas,International Calling Code,Mobile Prefix,Size of NSN,Carrier,Notes
0,Anguilla,True,1,264 772,10,NaN,NANP member
1,Antigua and Barbuda,True,1,268 7xx,10,NaN,NANP member
2,Argentina,True,54,15-Sep,?,"All carriers: Claro, Movistar, Personal, Tuenti",15 before the local number but after long dist...
3,Aruba,True,297,NaN,7,NaN,NaN
4,Bahamas,True,1,242 35x,10,NaN,NANP member
...,...,...,...,...,...,...,...
166,Venezuela,True,58,412,7,Digitel GSM,NaN
167,Venezuela,True,58,414,7,Movistar,NaN
168,Venezuela,True,58,416,7,Movilnet,NaN
169,Venezuela,True,58,424,7,Movistar,NaN


In [324]:
# Get country codes
input_countries = wh_phone_dict['Country']

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2

codes = [countries.get(country, 'Unknown code') for country in input_countries]

code_wh_phone_dict = wh_phone_dict
code_wh_phone_dict['Country Code'] = codes

In [325]:
# countries of interest with country codes
code_wh_phone_dict.head()

,Country,Americas,International Calling Code,Mobile Prefix,Size of NSN,Carrier,Notes,Country Code
0,Anguilla,True,1,264 772,10,NaN,NANP member,AI
1,Antigua and Barbuda,True,1,268 7xx,10,NaN,NANP member,AG
2,Argentina,True,54,15-Sep,?,"All carriers: Claro, Movistar, Personal, Tuenti",15 before the local number but after long dist...,AR
3,Aruba,True,297,NaN,7,NaN,NaN,AW
4,Bahamas,True,1,242 35x,10,NaN,NANP member,BS


In [326]:
# Manually spot check country codes and fix later
# code_wh_phone_dict[code_wh_phone_dict['Country Code'] == 'Unknown code'].Country.unique()

## Parse through phone numbers

In [327]:
phonenumbers.parse("+91 8320742364")

PhoneNumber(country_code=91, national_number=8320742364, extension=None, italian_leading_zero=None, number_of_leading_zeros=None, country_code_source=0, preferred_domestic_carrier_code=None)

In [328]:
sample_nums = pd.DataFrame(["+91 8320742364", "+918320742364", "918320742364", "+8320742364", "+1 5714993119"], 
                           columns=['Mobile'])

In [329]:
# Function to get country codes with a list of nums
l_ = []
for z in sample_nums['Mobile']:
    try:
        x = phonenumbers.parse(z)
        phone_country = (region_code_for_country_code(x.country_code))
        l_.append(phone_country)
    except:
        l_.append('Unknown')
        pass
sample_nums['Country Code'] = l_

In [330]:
sample_nums

,Mobile,Country Code
0,+91 8320742364,IN
1,+918320742364,IN
2,918320742364,Unknown
3,+8320742364,Unknown
4,+1 5714993119,US


## ISSUES
- handling missing country codes
- handling incorrectly formatted country codes

In [331]:
unknowns = sample_nums[sample_nums['Country Code'] == 'Unknown']
unknowns.head()

,Mobile,Country Code
2,918320742364,Unknown
3,+8320742364,Unknown


In [332]:
l_ = []
for i, r in unknowns.iterrows():
    num = r['Mobile']
    length = str(len(num))
    l_.append(length)
l_

unknowns['digitsCount'] = l_

<ipython-input-332-600c42431a9f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknowns['digitsCount'] = l_


In [333]:
unknowns

,Mobile,Country Code,digitsCount
2,918320742364,Unknown,12
3,+8320742364,Unknown,11


In [336]:
(code_wh_phone_dict[code_wh_phone_dict['Size of NSN'] == '11'].Notes)

15    Mobile phones use geographic area codes (two d...
Name: Notes, dtype: object